<a href="https://colab.research.google.com/github/hoon-72/hoon-72/blob/main/wav2vec2_korean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 설치

pip를 통해 설치를 진행합니다.


In [1]:
!pip install transformers
!pip install datasets
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.1 MB/s eta 0:00:00


# 2. 라이브러리 불러오기

wav2vec2 기반이라서 별도의 feature extraction을 필요로 하지 않습니다.

In [2]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset
import soundfile as sf
import torch
from jiwer import wer


In [3]:
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")

model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to('cuda')

ds = load_dataset("kresnik/zeroth_korean", "clean")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

# 3. 전체 데이터 중 Test set만을 이용함

파일 경로를 읽어서 raw signal을 저장합니다.

이어서 전체 test set에 대하여 결과를 result['transcription']에 저장합니다.

In [4]:
test_ds = ds['test']

In [5]:
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

test_ds = test_ds.map(map_to_array)

Map:   0%|          | 0/457 [00:00<?, ? examples/s]

In [6]:
def map_to_pred(batch):
    inputs = processor(batch["speech"], sampling_rate=16000, return_tensors="pt", padding="longest")
    input_values = inputs.input_values.to("cuda")
    #attention_mask = inputs.attention_mask.to("cuda")

    with torch.no_grad():
        #logits = model(input_values, attention_mask=attention_mask).logits
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    batch["transcription"] = transcription
    return batch

result = test_ds.map(map_to_pred, batched=True, batch_size=16, remove_columns=["speech"])

Map:   0%|          | 0/457 [00:00<?, ? examples/s]

In [7]:
print("WER:", wer(result["text"], result["transcription"]))

WER: 0.048034934497816595


In [8]:
print(result["text"])

['공정위는 이천 십 삼 년 유통산업발전법 개정을 근거로 이 공약이 이행된 것으로 평가했다', '이천 십 오 년에는 전국 모든 대학을 평가해 오 등급으로 나누고 등급에 따라 정원감축을 추진했다', '대한주택보증 입장에서는 손실 가능성을 최소화하기 위해 수수료를 높게 요구할 수밖에 없는 구조다', '공식 순방기간은 육 일부터 구 일까지였지만 이 후보자 부부는 이보다 앞선 사 일 함께 출국해 부인은 칠 일 이 후보자는 구 일 각각 귀국했다', '해수부는 선수 들기가 중단된 건 예측 할 수 없었던 너울 때문이며 장비 문제는 아니라는 입장입니다', '아무 자격과 전문성 없는 비선 최씨가 대외비인 대통령 연설문을 버젓이 받아보고 수정하는 등 국정을 농락한 사실부터 현실에서 일어나리라고 상상하기 어려웠던 일이다', '일각에서는 현역 의원의 안 의원 측근 그룹에 대한 견제로 보는 분석도 나왔다', '아이를 맡길 곳이 없는데 어린이집 측은 몇 달 간 옮길 곳을 찾았지만 주민들이 아이들 소음에 반대하는데다 까다로운 어린이집 설치기준을 맞출 방법이 없었다고 말합니다', '새로 길들이기 시작한 유익한 습관이 정착되기까지는 보통 시간이 걸리는데 수주에서 수개월이 걸리기도 합니다', '하지만 주인들은 사랑을 표현하고 싶은 자신의 욕구를 충족시키느라 애완동물을 돌보는 일에 푹 빠져 드는 것 같다', '목적은 다르지만 결국 모두 명품이 주는 보상심리에서 비롯됐다는 게 전문가들 분석이다', '친노계파가 인사권 전횡을 할 수 없는 그런 보직을 줘서는 안 되구요', '검거 당시 정씨 아들로 추정되는 이천 십 오 년생 아이도 있었다고 경찰은 전했다', '정씨는 금메달을 보여드려도 될까요라며 면접위원들에게 이 메달을 꺼내보였다고 한다', '신 총재는 일본의 과거사에 대한 사과와 배상 요구에 대해 그런 부분은 과거에 충분히 이야기가 다 됐던 부분이고 기본적으로 일본 천황이 공식적으로 사과를 했다', '그러면서 이 연구사는 위가 네 개나 되는 소의 소화과정을 보다 자세하게 설명했다', '이제 목표한

In [9]:
print(result["transcription"])

['공정위는 이천 십 삼 년 유통산업발전법 개정을 근거로 이 공약이 이행된 것으로 평가했다', '이천 십 오 년에는 전국 모든 대학을 평가해 오 등급으로 나누고 등급에 따라 정원감축을 추진했다', '대한주택보증 입장에서는 손실 가능성을 최소화하기 위해 수수료를 높게 요구할 수밖에 없는 구조다', '공식 순방기간은 육 일부터 구 일까지였지만 이 후보자 부부는 이보다 앞선 사 일 함께 출국해 부인은 칠 일 이 후보자는 구 일 각각 귀국했다', '해수부는 선수 들기가 존단된 건 렌측 할 수 없었던 올 때문이며 장비 문제는 아니라는 입장입니다', '아무 자격과 전성 없는 비선 최씨가 대외비인 대통령 여자문을 버젓이 받아보고 수정하는 등 걱정을 농락한 사실부터 현실에서 일어 나니라고 상상하기 어려웠던 일이다', '일각에서는 현역 의원의 안 의원 측근 그룹에 대한 견제로 보는 분석도 나왔다', '아이를 맡길 곳이 없는데 어린이집 측은 몇 달 간 옮길 곳을 찾았지만 주민들이 아이들 소음에 반대하는데다 까다로운 어린이집 설치기준을 맞출 방법이 없었다고 말합니다', '새로 길들이기 시작한 유익한 습관이 정착되기까지는 보통 시간이 걸리는데 수주에서 수개월이 걸리기도 합니다', '하지만 주인들은 사랑을 표현하고 싶은 자신의 욕구를 충족시키느라 애완동물들 돌보는 일에 푹 빠져 드는 것 같다', '목적은 다르지만 결국 모두 명품이 주는 보상심리에서 비롯됐다는 게 전문가들 분석이다', '친노계파가 인사권 전횡을 할 수 없는 그런 도직을 줘서는 안 되요', '검거 당시 정씨 아들로 추정되는 이천 십 오 년생 아이도 있었다고 경찰은 전했다', '정씨는 금메달을 보여드려도 될까요라며 면접위원들에게 이 메달을 꺼내보였다고 한다', '신 총재는 일본의 과거사에 대한 사과와 배상 요구에 대해 그런 부분은 과거에 충분히 이야기가아 됐던 부분이고 기본적으로 일본 천황이 공식적으로 사과를 했다', '그러면서 이 연구사는 위가 네 개나 되는 서의 소화과정을 보다 자세하게 설명했다', '이제 목표한 오백